# 8.4. Interactividad: ipywidgets.

- ipywidgets nos permite hacer de forma muy sencilla pequeñas aplicaciones dentro de los notebooks.
- Usaremos el decorador interact:  https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
- Puedes ver todos los widgets disponibles en: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
@interact(x=1.0)
def func_to_inter(x):
    print(x)

In [ ]:
@interact(x=True, y=1.0)
def func_to_inter(x, y):
    print(x, y)

- Puedes definir más concretamente el widget instanciandolo antes:

In [ ]:
@interact(
    x=widgets.IntSlider(min=-10, max=30, step=1, value=10),
    y=widgets.Dropdown(options=[('One', 1), ('Two', 2), ('Three', 3)], value=2,description='Number:')
)
def func_to_inter(x, y):
    print(x, y)

In [ ]:
@interact(
    a=widgets.FloatSlider(min=1, max=10, step=1, value=1),
    f=widgets.FloatSlider(min=1, max=50, step=1, value=1),
)
def func_to_inter(a, f):
    t = np.linspace(0, 1, 1000)
    y = a*np.sin(2*np.pi*f*t)  
    plt.plot(t, y)
    plt.ylim([-10, 10])